In [144]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [145]:
train = pd.read_csv('dataset-0510/train.csv')
test = pd.read_csv('dataset-0510/test.csv')

In [146]:
target = train['total_price']
train.drop('total_price', axis=1, inplace=True)

In [147]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)

In [148]:
df['village_income_median'].fillna(value=df.groupby(['city','town'])
                                   ['village_income_median'].transform('median'), 
                                   inplace=True)
print("groupby 之後還是有缺值：",len(df[df['village_income_median'].isnull()]))
df['village_income_median'].fillna(df['village_income_median'].mode(),inplace=True)

groupby 之後還是有缺值： 34


In [149]:
df['txn_floor'].fillna(df['total_floor'] // 2, inplace=True)
df['txn_floor'] = df['txn_floor'].astype('int64')

In [128]:
df.drop(['parking_area','parking_price'],axis=1,inplace=True)

In [130]:
def dummy_transform(df):
    categorical = []
    cat = ['total_floor','building_material',
       'city','building_type','txn_floor',
       'building_use','parking_way']
    categorical = cat + categorical
    numerical = [i for i in df.columns if i not in categorical]
    for col in categorical:
        df[col] = df[col].astype(str)
    df_dummy = pd.get_dummies(df[categorical])
    return pd.concat([df[numerical],df_dummy], axis=1) 

In [131]:
df = dummy_transform(df)

In [132]:
df.shape

(70000, 321)

In [133]:
df.drop(['building_material_5','txn_floor_28'], axis=1, inplace=True)

In [134]:
train = df.head(60000)
test = df.tail(10000)

In [137]:
y = np.log1p(target)
X = train.drop(['building_id'],axis=1)

In [138]:
X_valid = X[50000:]
y_valid = y[50000:]
X = X[:50000]
y = y[:50000]

In [140]:
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from mlxtend.regressor import StackingCVRegressor

from xgboost import XGBRegressor
from xgboost import plot_importance

In [141]:
def validation(prediction, ground_true):
    ground_true = np.floor(np.expm1(ground_true))
    ground_true = ground_true.reset_index(drop=True)
    
    prediction = np.floor(np.expm1(prediction))
    
    score = 0
    for i in range(len(prediction)):
        pred = prediction[i]
        roof = ground_true[i] + ground_true[i] / 10
        floor = ground_true[i] - ground_true[i] / 10
        if (pred <= roof) and (pred >= floor):
            score += 1
    return score

In [142]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=5000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,
    nthread=8,
    seed=41)

model.fit(
    X, 
    y, 
    eval_metric="rmse", 
    eval_set=[(X, y), (X_valid, y_valid)], 
    verbose=True, 
    early_stopping_rounds = 80)


[0]	validation_0-rmse:13.5616	validation_1-rmse:13.5563
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 80 rounds.
[1]	validation_0-rmse:12.2099	validation_1-rmse:12.2057
[2]	validation_0-rmse:10.9921	validation_1-rmse:10.9892
[3]	validation_0-rmse:9.8959	validation_1-rmse:9.89406
[4]	validation_0-rmse:8.90942	validation_1-rmse:8.90789
[5]	validation_0-rmse:8.02173	validation_1-rmse:8.02103
[6]	validation_0-rmse:7.22316	validation_1-rmse:7.22296
[7]	validation_0-rmse:6.50438	validation_1-rmse:6.5049
[8]	validation_0-rmse:5.85737	validation_1-rmse:5.85879
[9]	validation_0-rmse:5.27536	validation_1-rmse:5.27712
[10]	validation_0-rmse:4.75171	validation_1-rmse:4.75377
[11]	validation_0-rmse:4.28047	validation_1-rmse:4.2828
[12]	validation_0-rmse:3.85669	validation_1-rmse:3.85918
[13]	validation_0-rmse:3.4753	validation_1-rmse:3.47824
[14]	validation_0-rmse:3.13248	validation_1-rmse:3.13547


[137]	validation_0-rmse:0.231206	validation_1-rmse:0.246837
[138]	validation_0-rmse:0.230878	validation_1-rmse:0.24666
[139]	validation_0-rmse:0.230477	validation_1-rmse:0.246309
[140]	validation_0-rmse:0.230245	validation_1-rmse:0.246065
[141]	validation_0-rmse:0.230041	validation_1-rmse:0.24596
[142]	validation_0-rmse:0.229801	validation_1-rmse:0.245743
[143]	validation_0-rmse:0.229702	validation_1-rmse:0.24569
[144]	validation_0-rmse:0.229518	validation_1-rmse:0.245573
[145]	validation_0-rmse:0.229323	validation_1-rmse:0.245438
[146]	validation_0-rmse:0.229136	validation_1-rmse:0.245367
[147]	validation_0-rmse:0.229045	validation_1-rmse:0.24527
[148]	validation_0-rmse:0.228887	validation_1-rmse:0.245137
[149]	validation_0-rmse:0.228754	validation_1-rmse:0.245084
[150]	validation_0-rmse:0.228363	validation_1-rmse:0.244783
[151]	validation_0-rmse:0.227915	validation_1-rmse:0.244491
[152]	validation_0-rmse:0.227737	validation_1-rmse:0.244404
[153]	validation_0-rmse:0.227503	validation_

[274]	validation_0-rmse:0.209568	validation_1-rmse:0.233911
[275]	validation_0-rmse:0.209527	validation_1-rmse:0.233886
[276]	validation_0-rmse:0.209501	validation_1-rmse:0.233887
[277]	validation_0-rmse:0.209319	validation_1-rmse:0.233802
[278]	validation_0-rmse:0.209278	validation_1-rmse:0.233761
[279]	validation_0-rmse:0.209195	validation_1-rmse:0.233687
[280]	validation_0-rmse:0.209169	validation_1-rmse:0.233676
[281]	validation_0-rmse:0.209121	validation_1-rmse:0.233689
[282]	validation_0-rmse:0.209023	validation_1-rmse:0.233647
[283]	validation_0-rmse:0.208928	validation_1-rmse:0.233589
[284]	validation_0-rmse:0.208891	validation_1-rmse:0.233571
[285]	validation_0-rmse:0.208831	validation_1-rmse:0.233557
[286]	validation_0-rmse:0.208785	validation_1-rmse:0.233552
[287]	validation_0-rmse:0.208599	validation_1-rmse:0.233453
[288]	validation_0-rmse:0.208507	validation_1-rmse:0.233425
[289]	validation_0-rmse:0.208474	validation_1-rmse:0.233417
[290]	validation_0-rmse:0.208373	validat

[411]	validation_0-rmse:0.197034	validation_1-rmse:0.228481
[412]	validation_0-rmse:0.196979	validation_1-rmse:0.228447
[413]	validation_0-rmse:0.196933	validation_1-rmse:0.228417
[414]	validation_0-rmse:0.196792	validation_1-rmse:0.228365
[415]	validation_0-rmse:0.196745	validation_1-rmse:0.228322
[416]	validation_0-rmse:0.196667	validation_1-rmse:0.228285
[417]	validation_0-rmse:0.196615	validation_1-rmse:0.228264
[418]	validation_0-rmse:0.196587	validation_1-rmse:0.22825
[419]	validation_0-rmse:0.19655	validation_1-rmse:0.228254
[420]	validation_0-rmse:0.19647	validation_1-rmse:0.228219
[421]	validation_0-rmse:0.196283	validation_1-rmse:0.228147
[422]	validation_0-rmse:0.196245	validation_1-rmse:0.228154
[423]	validation_0-rmse:0.196198	validation_1-rmse:0.228122
[424]	validation_0-rmse:0.196078	validation_1-rmse:0.228056
[425]	validation_0-rmse:0.196062	validation_1-rmse:0.228044
[426]	validation_0-rmse:0.196022	validation_1-rmse:0.228036
[427]	validation_0-rmse:0.195937	validation

[549]	validation_0-rmse:0.187908	validation_1-rmse:0.224591
[550]	validation_0-rmse:0.187843	validation_1-rmse:0.224571
[551]	validation_0-rmse:0.187727	validation_1-rmse:0.224561
[552]	validation_0-rmse:0.187688	validation_1-rmse:0.224565
[553]	validation_0-rmse:0.187605	validation_1-rmse:0.22452
[554]	validation_0-rmse:0.187563	validation_1-rmse:0.224501
[555]	validation_0-rmse:0.187461	validation_1-rmse:0.22449
[556]	validation_0-rmse:0.187393	validation_1-rmse:0.224475
[557]	validation_0-rmse:0.187337	validation_1-rmse:0.224481
[558]	validation_0-rmse:0.187297	validation_1-rmse:0.224458
[559]	validation_0-rmse:0.187237	validation_1-rmse:0.22439
[560]	validation_0-rmse:0.18715	validation_1-rmse:0.224362
[561]	validation_0-rmse:0.187035	validation_1-rmse:0.224364
[562]	validation_0-rmse:0.186964	validation_1-rmse:0.22433
[563]	validation_0-rmse:0.186868	validation_1-rmse:0.224268
[564]	validation_0-rmse:0.186828	validation_1-rmse:0.224267
[565]	validation_0-rmse:0.186769	validation_1

...

[2717]	validation_0-rmse:0.110513	validation_1-rmse:0.209246
[2718]	validation_0-rmse:0.110462	validation_1-rmse:0.209225
[2719]	validation_0-rmse:0.110454	validation_1-rmse:0.209222
[2720]	validation_0-rmse:0.110422	validation_1-rmse:0.209236
[2721]	validation_0-rmse:0.110382	validation_1-rmse:0.209234
[2722]	validation_0-rmse:0.11036	validation_1-rmse:0.209245
[2723]	validation_0-rmse:0.110342	validation_1-rmse:0.209232
[2724]	validation_0-rmse:0.110332	validation_1-rmse:0.209218
[2725]	validation_0-rmse:0.11029	validation_1-rmse:0.209215
[2726]	validation_0-rmse:0.11026	validation_1-rmse:0.209236
[2727]	validation_0-rmse:0.110233	validation_1-rmse:0.209236
[2728]	validation_0-rmse:0.110202	validation_1-rmse:0.209248
[2729]	validation_0-rmse:0.11017	validation_1-rmse:0.209236
[2730]	validation_0-rmse:0.110141	validation_1-rmse:0.20923
[2731]	validation_0-rmse:0.110133	validation_1-rmse:0.209229
[2732]	validation_0-rmse:0.11012	validation_1-rmse:0.209233
[2733]	validation_0-rmse:0.110

[2852]	validation_0-rmse:0.107378	validation_1-rmse:0.208984
[2853]	validation_0-rmse:0.107341	validation_1-rmse:0.208995
[2854]	validation_0-rmse:0.107312	validation_1-rmse:0.208986
[2855]	validation_0-rmse:0.107307	validation_1-rmse:0.208986
[2856]	validation_0-rmse:0.107269	validation_1-rmse:0.208981
[2857]	validation_0-rmse:0.107242	validation_1-rmse:0.20897
[2858]	validation_0-rmse:0.107215	validation_1-rmse:0.208967
[2859]	validation_0-rmse:0.107207	validation_1-rmse:0.208958
[2860]	validation_0-rmse:0.107193	validation_1-rmse:0.208951
[2861]	validation_0-rmse:0.107179	validation_1-rmse:0.208946
[2862]	validation_0-rmse:0.107142	validation_1-rmse:0.208945
[2863]	validation_0-rmse:0.107133	validation_1-rmse:0.208954
[2864]	validation_0-rmse:0.107127	validation_1-rmse:0.208962
[2865]	validation_0-rmse:0.107109	validation_1-rmse:0.208967
[2866]	validation_0-rmse:0.107071	validation_1-rmse:0.208957
[2867]	validation_0-rmse:0.107041	validation_1-rmse:0.208941
[2868]	validation_0-rmse:

[2987]	validation_0-rmse:0.104558	validation_1-rmse:0.208882
[2988]	validation_0-rmse:0.104538	validation_1-rmse:0.208892
[2989]	validation_0-rmse:0.1045	validation_1-rmse:0.208886
[2990]	validation_0-rmse:0.104455	validation_1-rmse:0.208879
[2991]	validation_0-rmse:0.104426	validation_1-rmse:0.208871
[2992]	validation_0-rmse:0.104399	validation_1-rmse:0.208855
[2993]	validation_0-rmse:0.104383	validation_1-rmse:0.208856
[2994]	validation_0-rmse:0.104367	validation_1-rmse:0.20886
[2995]	validation_0-rmse:0.104352	validation_1-rmse:0.208871
[2996]	validation_0-rmse:0.104339	validation_1-rmse:0.208869
[2997]	validation_0-rmse:0.104329	validation_1-rmse:0.208871
[2998]	validation_0-rmse:0.10431	validation_1-rmse:0.208875
[2999]	validation_0-rmse:0.104295	validation_1-rmse:0.208882
[3000]	validation_0-rmse:0.104274	validation_1-rmse:0.208875
[3001]	validation_0-rmse:0.104261	validation_1-rmse:0.208871
[3002]	validation_0-rmse:0.10423	validation_1-rmse:0.208862
[3003]	validation_0-rmse:0.10

[3122]	validation_0-rmse:0.101847	validation_1-rmse:0.208525
[3123]	validation_0-rmse:0.101834	validation_1-rmse:0.208525
[3124]	validation_0-rmse:0.101821	validation_1-rmse:0.208528
[3125]	validation_0-rmse:0.10179	validation_1-rmse:0.208512
[3126]	validation_0-rmse:0.101782	validation_1-rmse:0.208512
[3127]	validation_0-rmse:0.101751	validation_1-rmse:0.208532
[3128]	validation_0-rmse:0.101724	validation_1-rmse:0.208517
[3129]	validation_0-rmse:0.101672	validation_1-rmse:0.208514
[3130]	validation_0-rmse:0.101655	validation_1-rmse:0.208516
[3131]	validation_0-rmse:0.10163	validation_1-rmse:0.208524
[3132]	validation_0-rmse:0.101589	validation_1-rmse:0.208517
[3133]	validation_0-rmse:0.101565	validation_1-rmse:0.208518
[3134]	validation_0-rmse:0.101542	validation_1-rmse:0.208514
[3135]	validation_0-rmse:0.101518	validation_1-rmse:0.208517
[3136]	validation_0-rmse:0.101491	validation_1-rmse:0.208516
[3137]	validation_0-rmse:0.101483	validation_1-rmse:0.208513
[3138]	validation_0-rmse:0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eta=0.3, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=8,
       min_child_weight=300, missing=None, n_estimators=5000, n_jobs=1,
       nthread=8, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=41, silent=True,
       subsample=0.8)

In [143]:
validation(model.predict(X_valid),y_valid)

4938